# Evaluation data

In [10]:
import requests
import pandas as pd

url_prefix = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/03-evaluation/'
docs_url = url_prefix + 'search_evaluation/documents-with-ids.json'
documents = requests.get(docs_url).json()

ground_truth_url = url_prefix + 'search_evaluation/ground-truth-data.csv'
df_ground_truth = pd.read_csv(ground_truth_url)
ground_truth = df_ground_truth.to_dict(orient='records')

In [34]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [35]:
ground_truth[0]

{'question': 'When does the course begin?',
 'course': 'data-engineering-zoomcamp',
 'document': 'c02e79ef'}

In [11]:
from tqdm.auto import tqdm

def hit_rate(relevance_total):
    cnt = 0

    for line in relevance_total:
        if True in line:
            cnt = cnt + 1

    return cnt / len(relevance_total)

def mrr(relevance_total):
    total_score = 0.0

    for line in relevance_total:
        for rank in range(len(line)):
            if line[rank] == True:
                total_score = total_score + 1 / (rank + 1)

    return total_score / len(relevance_total)

def evaluate(ground_truth, search_function):
    relevance_total = []

    for q in tqdm(ground_truth):
        doc_id = q['document']
        results = search_function(q)
        relevance = [d['id'] == doc_id for d in results]
        relevance_total.append(relevance)

    return {
        'hit_rate': hit_rate(relevance_total),
        'mrr': mrr(relevance_total),
    }

# LLM

In [8]:
import os
import yaml
from openai import AzureOpenAI

In [9]:
from dataclasses import dataclass

@dataclass
class LLMServiceConfig:
    host: str
    api_key: str
    api_version: str
    model: str

with open("../OPENAI_API_KEY.yaml") as f:
    details = yaml.safe_load(f)["Glossary Terms Extraction Service"]
    
service_config = LLMServiceConfig(
    host=f"{details['protocol']}://{details['host']}",
    api_key=details['api_key'],
    api_version=details['api_version'],
    model=details['model']
)
client = AzureOpenAI(
    api_version=service_config.api_version,
    azure_endpoint=service_config.host,
    api_key=service_config.api_key
)

# Q1 

In [14]:
import minsearch

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp',
 'id': 'c02e79ef'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "section", "text"],
    keyword_fields=["course", "id"]
)

index.fit(documents)

In [17]:
def minsearch_search(query, course):
    boost = {'question': 1.5, 'section': 0.1}
    
    results = index.search(
        query=query,
        filter_dict={'course': course},
        boost_dict=boost,
        num_results=5
    )

    return results

In [90]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    results = minsearch_search(query=q['question'], course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4627/4627 [00:05<00:00, 865.25it/s]


In [91]:
f"Hitrate is: {hit_rate(relevance_total)}"

'Hitrate is: 0.848714069591528'

In [92]:
mrr(relevance_total)

0.7288235717887772

# Embeddings

In [105]:
from minsearch import VectorSearch

In [106]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.pipeline import make_pipeline

In [107]:
texts = []

for doc in documents:
    t = doc['question']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

# Q2

In [108]:
vindex = VectorSearch(keyword_fields={'course'})
vindex.fit(X, documents)

In [110]:
def minsearch_vindex_search(query_vector, course):
    
    results = vindex.search(
        query_vector=query_vector,
        filter_dict={'course': course},
        num_results=5
    )

    return results

In [112]:
doc_id = q['document']
query = q['question']
queries_vector = pipeline.transform([query])[0]
results = minsearch_vindex_search(query_vector=queries_vector, course=q['course'])
relevance = [d['id'] == doc_id for d in results]
relevance

[{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.", 'section': 'General course-related questions', 'question': 'Course - When will the course start?', 'course': 'data-engineering-zoomcamp', 'id': 'c02e79ef'}, {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engin

[True, False, False, False, False]

In [113]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query = q['question']
    queries_vector = pipeline.transform([query])[0]
    results = minsearch_vindex_search(query_vector=queries_vector, course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|████████████████████████████████████████████████████████████████████████| 4627/4627 [00:02<00:00, 1774.87it/s]


In [115]:
f"MRR is: {mrr(relevance_total)}"

'MRR is: 0.35720048987825087'

# Q3

In [121]:
texts = []

for doc in documents:
    t = doc['question'] + ' ' + doc['text']
    texts.append(t)

pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)
X = pipeline.fit_transform(texts)

In [122]:
vindex2 = VectorSearch(keyword_fields={'course'})
vindex2.fit(X, documents)

In [123]:
def minsearch_vindex2_search(query_vector, course):
    
    results = vindex2.search(
        query_vector=query_vector,
        filter_dict={'course': course},
        num_results=5
    )

    return results

In [124]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query = q['question']
    queries_vector = pipeline.transform([query])[0]
    results = minsearch_vindex2_search(query_vector=queries_vector, course=q['course'])
    relevance = [d['id'] == doc_id for d in results]
    relevance_total.append(relevance)

100%|████████████████████████████████████████████████████████████████████████| 4627/4627 [00:03<00:00, 1295.44it/s]


In [126]:
f"MRR is: {hit_rate(relevance_total)}"

'MRR is: 0.8210503566025502'

# Q4

In [182]:
from qdrant_client import QdrantClient, models

client = QdrantClient("http://localhost:6333")
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='zoomcamp-rag'), CollectionDescription(name='zoomcamp-hw3_with_ids'), CollectionDescription(name='zoomcamp-hw3'), CollectionDescription(name='zoomcamp-hw2')])

In [183]:
model_handle = "jinaai/jina-embeddings-v2-small-en"
EMBEDDING_DIMENSIONALITY = 512
limit = 5

In [184]:
from fastembed import TextEmbedding
[el for el in TextEmbedding.list_supported_models() if el['model'] == model_handle]

[{'model': 'jinaai/jina-embeddings-v2-small-en',
  'sources': {'hf': 'xenova/jina-embeddings-v2-small-en',
   'url': None,
   '_deprecated_tar_struct': False},
  'model_file': 'onnx/model.onnx',
  'description': 'Text embeddings, Unimodal (text), English, 8192 input tokens truncation, Prefixes for queries/documents: not necessary, 2023 year.',
  'license': 'apache-2.0',
  'size_in_GB': 0.12,
  'additional_files': [],
  'dim': 512,
  'tasks': {}}]

In [187]:
# Define the collection name
collection_name = "zoomcamp-hw3"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    ),
        
)

True

In [188]:
points = []
id = 0

for doc in documents:
    text = doc['question'] + ' ' + doc['text']

    point = models.PointStruct(
        id=id,
        vector=models.Document(text=text, model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
        payload={
            "text": doc['text'],
            "section": doc['section'],
            "course": doc['course'],
            "id": doc['id']
        } #save all needed metadata fields
    )
    points.append(point)

    id += 1

In [189]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [191]:
def qdrant_search(query, course, limit=5):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        query_filter=models.Filter(# filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True, #to get metadata in the results
    )

    return results

In [193]:
q = ground_truth[0]
doc_id = q['document']
query = q['question']
results = qdrant_search(query=query, course=q['course'])
relevance = [d.payload['id'] == doc_id for d in results.points]
relevance

[True, False, False, False, False]

In [196]:
relevance_total = []

for q in tqdm(ground_truth):
    doc_id = q['document']
    query = q['question']
    results = qdrant_search(query=query, course=q['course'])
    relevance = [d.payload['id'] == doc_id for d in results.points]
    relevance_total.append(relevance)

100%|█████████████████████████████████████████████████████████████████████| 4627/4627 [00:36<00:00, 125.28it/s]


In [197]:
f"MRR is: {hit_rate(relevance_total)}"

'MRR is: 0.9299762264966501'

# Q5

In [200]:
results_url = url_prefix + 'rag_evaluation/data/results-gpt4o-mini.csv'
df_results = pd.read_csv(results_url)
df_results.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [199]:
pipeline = make_pipeline(
    TfidfVectorizer(min_df=3),
    TruncatedSVD(n_components=128, random_state=1)
)

In [201]:
pipeline.fit(df_results.answer_llm + ' ' + df_results.answer_orig + ' ' + df_results.question)

,steps,"[('tfidfvectorizer', ...), ('truncatedsvd', ...)]"
,transform_input,None
,memory,None
,verbose,False
,input,'content'
,encoding,'utf-8'
,decode_error,'strict'
,strip_accents,None
,lowercase,True
,preprocessor,None
,tokenizer,None


In [217]:
import numpy as np

def cosine(u, v):
    u_norm = np.sqrt(u.dot(u))
    v_norm = np.sqrt(v.dot(v))
    return u.dot(v) / (u_norm * v_norm)

In [218]:
rec = df_results.to_dict(orient='records')[0]
v_llm, v_orig = pipeline.transform([rec['answer_llm'], rec['answer_orig']])
cosine(v_llm, v_orig)

np.float64(0.46352620160029917)

In [221]:
records = df_results.to_dict(orient='records')
res = []
for rec in records:
    v_llm, v_orig = pipeline.transform([rec['answer_llm'], rec['answer_orig']])
    res.append(cosine(v_llm, v_orig)) 

In [224]:
f"Average cosine similarity is: {np.average(res)}" 

'Average cosine similarity is: 0.8415841233490402'

# Q6

In [225]:
!pip install rouge

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [226]:
df_results.iloc[10]

answer_llm     Yes, all sessions are recorded, so if you miss...
answer_orig    Everything is recorded, so you won’t miss anyt...
document                                                5170565b
question                    Are sessions recorded if I miss one?
course                                 machine-learning-zoomcamp
Name: 10, dtype: object

In [227]:
from rouge import Rouge
rouge_scorer = Rouge()

r = df_results.iloc[10]
scores = rouge_scorer.get_scores(r.answer_llm, r.answer_orig)[0]
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

In [228]:
scores = []
for idx, row in df_results.iterrows():
    score = rouge_scorer.get_scores(row['answer_llm'], row['answer_orig'])[0]['rouge-1']['f']
    scores.append(score)

In [229]:
f"Average Rouge-1 F1 is: {np.average(scores)}" 

'Average Rouge-1 F1 is: 0.3516946452113943'